In [1]:
import logging
from utilenames import tilenames_mkd, tilenames_tls, tilenames_rgn
from os.path import join, isfile
from os import makedirs
from glob import glob
from uinterp import riofill
from ufuncs import get_raster_info, gdal_regrid
from uvars import gdsm_v_fn, gdtm_v_fn, egm08_fn, outdir, indir
import sys
from uvars import topoxcale_dir
sys.path.append(topoxcale_dir)
from topoxcale.mlxcale import mldownxcale
from topoxcale.sagaxcale import gwrdownxcale
import time

do the better naming of the data, and clean the functions 
do the filtering FAST and optimized (1h budget)

In [2]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("processing.log"), # add date to this 
        logging.StreamHandler(sys.stdout)
    ]
)

ti = time.perf_counter()
#tilenames #N13E103 #'N10E105','S01W063'
logging.info('Loading main variables')
#tilenames = ['N13E103']#tilenames_mkd + tilenames_tls + tilenames_rgn
tilenames = tilenames_mkd + tilenames_tls + tilenames_rgn
varname = "tdem_dem" # use EDEM #deps: filling methos for my DEMs
roitilenames = tilenames
makedirs(outdir, exist_ok=True)
xres, yres = 0.01057350068885258912, -0.01057350068885258912
t_epsg = 'EPSG:4326'
mode = "num"

varpath = f"{indir}/*/*{varname}.tif"
varfiles = glob(varpath)
roivarfiles = [fi for fi in varfiles for t in roitilenames if t in fi]

ta = time.perf_counter()
gdsmf_tiles = []
gdtmf_tiles = []
geoid_tiles = []
logging.info(f'setting baseline dem as {varname}')

2025-05-01 14:46:45,513 - INFO - Loading main variables
2025-05-01 14:46:45,523 - INFO - setting baseline dem as tdem_dem


In [3]:
si = 0 
for fi in roivarfiles:
    tilename = fi.split('/')[-2]
    tile_dir = join(outdir, 'TILES', tilename)
    makedirs(tile_dir, exist_ok=True)
    logging.info(f"Processing tile: {tilename}")
    _, _, _, xmin, xmax, ymin, ymax, _, _ = get_raster_info(fi)
    
    gdsmv_tile = f"{tile_dir}/{tilename}_gdsm_void.tif"
    gdtmv_tile = f"{tile_dir}/{tilename}_gdtm_void.tif"
    geoid_tile = f"{tile_dir}/{tilename}_egm08.tif"
    logging.info(f'regriding GDSM @{tilename}...')
    gdal_regrid(gdtm_v_fn, gdtmv_tile, xmin, ymin, xmax, ymax, xres, yres, mode, t_epsg, overwrite=False)
    logging.info(f'regriding GDTM  @{tilename}...')
    gdal_regrid(gdsm_v_fn, gdsmv_tile, xmin, ymin, xmax, ymax, xres, yres, mode, t_epsg, overwrite=False)

    logging.info(f'regriding GEOID  @{tilename}...')
    gdal_regrid(egm08_fn, geoid_tile, xmin, ymin, xmax, ymax, xres, yres, mode, t_epsg, overwrite=False)

    gdsmf_tile = gdsmv_tile.replace('void.tif', f'riofill{si}.tif')
    gdtmf_tile = gdtmv_tile.replace('void.tif', f'riofill{si}.tif')

    logging.info(f'tFilling GDSM @{tilename}...')
    riofill(gdtmv_tile, gdtmf_tile, si=si)
    logging.info(f'tFilling GDSM @{tilename}...')
    riofill(gdsmv_tile, gdsmf_tile, si=si)
    
    gdsmf_tiles.append(gdsmf_tile)
    gdtmf_tiles.append(gdtmf_tile)
    geoid_tiles.append(geoid_tile)

tb = time.perf_counter()
logging.info(f'Void filling time: {(tb - ta)/60:.2f} min')

2025-05-01 14:46:46,369 - INFO - Processing tile: N09E105
2025-05-01 14:46:46,392 - INFO - regriding GDSM @N09E105...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,466 - INFO - regriding GDTM  @N09E105...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,529 - INFO - regriding GEOID  @N09E105...
Source NoData Value: -32767.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,599 - INFO - tFilling GDSM @N09E105...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E105/N09E105_gdtm_riofill0.tif
2025-05-01 14:46:46,601 - INFO - tFilling GDSM @N09E105...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E105/N09E105_gdsm_riofill0.tif
2025-05-01 14:46:46,603 - INFO - Processing tile: N09E106
2025-05-01 14:46:46,606 - INFO - regriding GDSM @N09E106...


ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E105/N09E105_gdtm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E105/N09E105_gdsm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E105/N09E105_egm08.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,673 - INFO - regriding GDTM  @N09E106...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,740 - INFO - regriding GEOID  @N09E106...
Source NoData Value: -32767.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,805 - INFO - tFilling GDSM @N09E106...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E106/N09E106_gdtm_riofill0.tif
2025-05-01 14:46:46,807 - INFO - tFilling GDSM @N09E106...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E106/N09E106_gdsm_riofill0.tif
2025-05-01 14:46:46,809 - INFO - Processing tile: N10E104
2025-05-01 14:46:46,812 - INFO - regriding GDSM @N10E104...


ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E106/N09E106_gdtm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E106/N09E106_gdsm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N09E106/N09E106_egm08.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,879 - INFO - regriding GDTM  @N10E104...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:46,937 - INFO - regriding GEOID  @N10E104...
Source NoData Value: -32767.0
Destination NoData Value: -9999.0
2025-05-01 14:46:47,007 - INFO - tFilling GDSM @N10E104...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N10E104/N10E104_gdtm_riofill0.tif
2025-05-01 14:46:47,009 - INFO - tFilling GDSM @N10E104...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N10E104/N10E104_gdsm_riofill0.tif
2025-05-01 14:46:47,011 - INFO - Processing tile: N10E105
2025-05-01 14:46:47,014 - INFO - regriding GDSM @N10E105...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0


...

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S01W063/S01W063_egm08.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S01W064/S01W064_gdtm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S01W064/S01W064_gdsm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S01W064/S01W064_egm08.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:49,454 - INFO - regriding GDTM  @S02W063...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:49,515 - INFO - regriding GEOID  @S02W063...
Source NoData Value: -32767.0
Destination NoData Value: -9999.0
2025-05-01 14:46:49,577 - INFO - tFilling GDSM @S02W063...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W063/S02W063_gdtm_riofill0.tif
2025-05-01 14:46:49,579 - INFO - tFilling GDSM @S02W063...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W063/S02W063_gdsm_riofill0.tif
2025-05-01 14:46:49,581 - INFO - Processing tile: S02W064
2025-05-01 14:46:49,584 - INFO - regriding GDSM @S02W064...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:49,646 - INFO - regriding GDTM  @S02W064...


ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W063/S02W063_gdtm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W063/S02W063_gdsm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W063/S02W063_egm08.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W064/S02W064_gdtm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



Source NoData Value: -9999.0
Destination NoData Value: -9999.0
2025-05-01 14:46:49,707 - INFO - regriding GEOID  @S02W064...
Source NoData Value: -32767.0
Destination NoData Value: -9999.0
2025-05-01 14:46:49,776 - INFO - tFilling GDSM @S02W064...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W064/S02W064_gdtm_riofill0.tif
2025-05-01 14:46:49,777 - INFO - tFilling GDSM @S02W064...
already created /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W064/S02W064_gdsm_riofill0.tif
2025-05-01 14:46:49,779 - INFO - Void filling time: 0.07 min


ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W064/S02W064_gdsm_void.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.

ERROR 1: Output dataset /media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/S02W064/S02W064_egm08.tif exists,
but some command line options were provided indicating a new dataset
should be created.  Please delete existing dataset and run again.



In [4]:
from ugeoid import ellipsoid2orthometric
from ufuncs import calculate_dod,min_rasters

In [ ]:
for i in range(len(gdsmf_tiles)):
    print(i)

    gdsm_tile = gdsmf_tiles[i]
    gdsm_tile_egm = gdsm_tile.replace('.tif', '_egm.tif')

    gdtm_tile = gdtmf_tiles[i]
    gdtm_tile_egm = gdtm_tile.replace('.tif', '_egm.tif')

    geoid_tile = geoid_tiles[i]
    dod_tile = gdsm_tile.replace('gdsm_riofill0', 'dod')
    dod_tile_egm = gdsm_tile_egm.replace('gdsm_riofill0', 'dod')

    ellipsoid2orthometric(gdsm_tile,geoid_tile,gdsm_tile_egm)
    ellipsoid2orthometric(gdtm_tile,geoid_tile,gdtm_tile_egm)

    _ = calculate_dod(gdtm_tile_egm, gdsm_tile_egm, dod_tile_egm)
    _ = calculate_dod(gdtm_tile, gdsm_tile, dod_tile)

    varname = "edem_egm"#tdem_dem
    varpath = f"{indir}/*/*{varname}.tif"
    varfiles = glob(varpath)
    roivarfiles = [fi for fi in varfiles for t in roitilenames if t in fi]
    roivarfiles

    sfix = "GWR"
    logging.info(f'Style Transfer Started using {sfix}...')
    xpath = roivarfiles[i] #gdtm_tile_egm,gdtm_tile
    ypath = gdtm_tile_egm
    out_path = ypath.replace('.tif', f'_{sfix}.tif')
    if not isfile(out_path):
        logging.info(f"GWR downscaling (gdtm): {tilename}")
        gwrdownxcale(xpath, ypath, out_path, oaux=False, epsg_code=4979, clean=False)
    tc = time.perf_counter()
    logging.info(f'GWR downscaling (gdtm) time: {(tc - tb)/60:.2f} min')

    xpath = roivarfiles[i] #gdtm_tile_egm,gdtm_tile
    out_path = ypath.replace('.tif', f'_{sfix}.tif')
    ps_tile = out_path.replace(f'_{sfix}.tif', '_fmin.tif')
    min_rasters(xpath, out_path, ps_tile)



0
2025-05-01 14:46:53,196 - INFO - Style Transfer Started using GWR...
2025-05-01 14:46:53,197 - INFO - GWR downscaling (gdtm): S02W064


____________________________

   #####   ##   #####    ##
  ###     ###  ##       ###
   ###   # ## ##  #### # ##
    ### ##### ##    # #####
 ##### #   ##  ##### #   ##
____________________________

SAGA Version: 8.2.2

____________________________
library path: /usr/lib/x86_64-linux-gnu/saga/
library name: libstatistics_regression
library     : statistics_regression
tool        : GWR for Grid Downscaling
identifier  : 14
author      : O.Conrad (c) 2013
processors  : 56 [56]
____________________________

loading: N09E105_edem_egm

  1%